In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image 
import os
import random
import cv2

import keras 
from sklearn.model_selection import train_test_split 
from tensorflow.keras import utils

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Dense, Dropout 
from keras.layers import Flatten, BatchNormalization

In [ ]:
!pip install -U six numpy wheel
!pip install -U keras_preprocessing --no-deps

In [ ]:
#read_file = pd.read_excel(r'C:\Users\anndz\Desktop\BP\COVID.metadata.xlsx')
#read_file.to_csv(r'C:\Users\anndz\Desktop\BP\COVID.metadata.csv', index = None, header=True)
#COVID = read_file

Predspracovanie

In [2]:
Ycat = []
Ybin = []
#labels = ['Normal', 'Lung_Opacity','Viral Pneumonia','Tuberculosis']
labels = ['Normal', 'Lung_Opacity','Viral_Pneumonia','Tuberculosis','COVID']
size = 256

def get_data(folder, Ycat, Ybin):
    x = 0
    data = []
    for label in labels: 
        path = os.path.join(folder, label)
        for images in os.listdir(path):
            img = cv2.imread(os.path.join(folder,label, images))[...,::-1]
            img = cv2.resize(img, (size, size))
            img = img.astype('float16')
            img /= 255
            data.append(img)
            Ycat.append(label)
            if label == 'COVID':
                Ybin.append(1)
            else:
                Ybin.append(0)
        x = x + 1
    return data, cx

In [3]:
Xdata = get_data('',Ycat, Ybin)

In [ ]:
plt.imshow(Xdata[1], cmap='gray')
print(Ycat[1],Ybin[1])

In [4]:
#Xdata = Xdata.sample(n=len(Xdata), random_state=1242)
#Ycat = Ycat.sample(n=len(Ycat), random_state=1242)
random.seed(151)
Xdata = random.sample(Xdata,3600)
random.seed(151)
Ycat=random.sample(Ycat,3600)
random.seed(151)
Ybin=random.sample(Ybin,3600)
X = np.array(Xdata)
Ycat = np.array(Ycat)
Ybin = np.array(Ybin)

In [ ]:
#Ycat = utils.to_categorical(labels,5)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, Ybin, test_size=0.3, random_state=1)

In [20]:
print(x_train.shape, x_test.shape,y_train.shape, y_test.shape)

(2520, 256, 256, 3) (1080, 256, 256, 3) (2520,) (1080,)


In [17]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(256,256,3)))

model.add(MaxPooling2D(pool_size=(2,2))),

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 516128)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               66064512  
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 5)                 645       
                                                      

In [18]:
from tensorflow.keras.optimizers import Adam

adam = Adam()
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
f = model.fit(x_train, y_train, epochs = 3,batch_size=128, validation_data=(x_test, y_test))